In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
import os
from shutil import copyfile

#for dirname, _, filenames in os.walk('/kaggle/input'):
   #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
source_training_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train/'
source_test_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test/'

pred_dir = '/kaggle/input/intel-image-classification/seg_train/seg_pred/'

train_path = '/kaggle/temp/seg_train/'
test_path = '/kaggle/temp/seg_test/'




In [3]:

def count_files_in_folder(source_dir, new_path):
    
    #remove files with zero lenght and save in the new dirrectory 
    zero_length_files = []
    for dirr in os.listdir(source_dir):
        new_subdir = os.path.join(new_path, dirr)
        os.makedirs(new_subdir)
        for img in os.listdir(os.path.join(source_dir, dirr)):
            if os.path.getsize(os.path.join(source_dir, dirr, img)) !=0:
                copyfile(os.path.join(source_dir, dirr, img), os.path.join(new_subdir,img))
            else:
                zero_length_files.append(img)
                print(f"{img} is zero length, so ignoring")
        
    total_files = 0
    #Walk through all directories and subdirectories
    for root, dirs, files in os.walk(new_path):
        total_files += len(files)
    #directories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]
    #return directories
    if len(zero_length_files) == 0:
        print("No zero length files found.")

    return (total_files)

In [4]:
training_files = count_files_in_folder(source_training_dir,train_path)
test_files = count_files_in_folder(source_test_dir,test_path)

print(f"There are {training_files} files in the training set")
print(f"there are {test_files} files in the testing set")

No zero length files found.
No zero length files found.
There are 14034 files in the training set
there are 3000 files in the testing set


In [5]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()